In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2
import time

In [2]:
pred_path='Infence Images'

In [3]:
classes = [
    'Bacterial_spot',
    'Black_mold',
    'Early_blight',
    'Healthy',
    'Late_blight',
    'Mosaic_virus',
    'Septoria_spot',
]

In [4]:
model = torch.load('Exported Model/resnext50_32x4d_final.pth')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [5]:
mean = [0.4363, 0.4328, 0.3291]
std = [0.2129, 0.2075, 0.2038]

transformer = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [6]:
def prediction(img_path,transformer):
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]

    return pred

In [7]:
images_path=glob.glob(pred_path+'/*.jpg')

In [8]:
pred_dict={}

since =time.time()

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
    
time_elapsed = time.time() - since
print('Inference complete in {:.0f}ms'.format(
        time_elapsed * 1000))

Inference complete in 3869ms


In [9]:
pred_dict

{'Infence Images\\BM_1.jpg': 'Black_mold',
 'Infence Images\\BM_2.jpg': 'Black_mold',
 'Infence Images\\BM_3.jpg': 'Black_mold',
 'Infence Images\\BM_4.jpg': 'Early_blight',
 'Infence Images\\BM_5.jpg': 'Black_mold',
 'Infence Images\\BS_1.jpg': 'Bacterial_spot',
 'Infence Images\\BS_2.jpg': 'Early_blight',
 'Infence Images\\BS_3.jpg': 'Bacterial_spot',
 'Infence Images\\BS_4.JPG': 'Septoria_spot',
 'Infence Images\\BS_5.jpg': 'Bacterial_spot',
 'Infence Images\\EB_1.jpg': 'Septoria_spot',
 'Infence Images\\EB_2.jpg': 'Early_blight',
 'Infence Images\\EB_3.jpg': 'Early_blight',
 'Infence Images\\EB_4.jpg': 'Early_blight',
 'Infence Images\\EB_5.jpg': 'Early_blight',
 'Infence Images\\H_1.jpg': 'Healthy',
 'Infence Images\\H_2.jpg': 'Mosaic_virus',
 'Infence Images\\H_3.jpg': 'Healthy',
 'Infence Images\\H_4.jpg': 'Healthy',
 'Infence Images\\H_5.jpg': 'Mosaic_virus',
 'Infence Images\\LB_1.jpg': 'Late_blight',
 'Infence Images\\LB_2.jpg': 'Early_blight',
 'Infence Images\\LB_3.jpg': 'L